# Import Necessary Library

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import math
import os
import urllib.request
from functools import partial
from urllib.error import HTTPError

# What is Attention?

Attention in neural networks, particularly relevant for sequential tasks, refers to a mechanism that selectively focuses on certain parts of input data. This concept has gained significant interest in recent years. In essence, attention computes a weighted average of elements in a sequence, with the weights being dynamically determined based on the relevance of each element to a specific query. This allows the model to prioritize certain inputs over others.

The attention mechanism consists of four primary components:

* **Query**: A feature vector representing the target of the attention, essentially indicating the information the model seeks within the sequence.
* **Keys**: Feature vectors corresponding to each input element, describing the content or relevance of the elements. The keys help the model identify which elements to focus on, relative to the query.
* **Values**: Feature vectors representing the actual content from each input element that the model should aggregate.
* **Score function**: A function used to calculate attention weights, representing the relevance of each key-query pair. Common implementations include simple operations like the dot product or more complex structures like a small neural network.

The attention mechanism operates by first computing scores between the query and each key using the score function. These scores determine the attention weights through a softmax function, ensuring that they sum to one and are non-negative. The output is then calculated as the weighted sum of the value vectors, with weights corresponding to the calculated attention scores.

Mathematically, this process can be represented as:

$$
\alpha_i = \frac{\exp\left(f_{attn}\left(\text{key}_i, \text{query}\right)\right)}{\sum_j \exp\left(f_{attn}\left(\text{key}_j, \text{query}\right)\right)}, \hspace{5mm} \text{out} = \sum_i \alpha_i \cdot \text{value}_i
$$

In practice, attention mechanisms can vary based on the choice of queries, the definition of key and value vectors, and the specific score function used. A prominent example is the **self-attention** mechanism used in the Transformer architecture, where each element in a sequence provides its own key, value, and query. The self-attention mechanism allows each element to attend to all elements in the sequence, including itself, resulting in a representation that incorporates information from the entire sequence.

The above explanation provides a conceptual understanding of the attention mechanism, highlighting its components and operational principles without delving into the specific details of any particular implementation, such as the scaled dot product attention used in Transformers.

### Scaled Dot Product Attention

The scaled dot product attention is a fundamental component of the self-attention mechanism, enabling elements within a sequence to efficiently attend to one another. It operates on queries $Q\in\mathbb{R}^{T\times d_k}$, keys $K\in\mathbb{R}^{T\times d_k}$, and values $V\in\mathbb{R}^{T\times d_v}$, where $T$ represents the sequence length and $d_k$, $d_v$ denote the dimensions of queries/keys and values, respectively.

The mechanism calculates the attention values based on the dot product similarity between each query $Q_i$ and key $K_j$, and scales the results by the square root of the dimensionality of the keys, $d_k$. The formula for this calculation is:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Here, the matrix product $QK^T$ computes the dot product between all pairs of queries and keys, forming a $T\times T$ matrix where each entry represents the attention score from one element to another. After applying the softmax function, these scores are used as weights to compute a weighted average of the value vectors.

The scaling factor $1/\sqrt{d_k}$ is critical for maintaining the variance of the attention scores at an appropriate level. Without this scaling, the variance of the dot products could become too large, leading to a situation where the softmax function saturates, with most of its output concentrated on a single element. This would hinder learning by resulting in gradients that are almost zero.

Additionally, the mechanism can include an optional masking step (denoted as `Mask (opt.)` in the diagram), useful in situations like batch processing of sequences of varying lengths. Padding is used to equalize the lengths of sequences, and the mask ensures that the padded positions do not affect the attention calculation, typically by assigning a very low value to these positions in the attention scores.

In summary, the scaled dot product attention efficiently enables each element in a sequence to attend to all others, considering the relevance of each element, and is crucial for models that rely on self-attention, such as Transformers.

### Implementing Scaled Dot Product Attention

Scaled dot product attention is a core mechanism allowing each element in a sequence to consider all other elements efficiently, which is fundamental in self-attention models like Transformers. Here's a detailed guide to implementing scaled dot product attention, breaking down the components and the steps involved.

#### Inputs to the Attention Mechanism
The attention function takes three inputs:
1. **Queries (Q)**: $Q\in\mathbb{R}^{T\times d_k}$, where $T$ is the sequence length and $d_k$ is the dimensionality of the queries and keys.
2. **Keys (K)**: $K\in\mathbb{R}^{T\times d_k}$.
3. **Values (V)**: $V\in\mathbb{R}^{T\times d_v}$, where $d_v$ is the dimensionality of the values.

#### Step-by-Step Calculation
1. **Dot Product of Queries and Keys**: Calculate the dot product between each query and all keys to obtain a measure of compatibility or relevance between each query-key pair. This results in a matrix of shape $T \times T$, where each element $(i, j)$ represents the dot product between query $i$ and key $j$.
   
   $$\text{Score Matrix} = QK^T$$

2. **Scaling**: Scale the scores obtained in the previous step by dividing by $\sqrt{d_k}$ to ensure stable gradients, as larger values of $d_k$ can lead to extremely small gradients, which can slow down learning and model convergence.

   $$\text{Scaled Score Matrix} = \frac{\text{Score Matrix}}{\sqrt{d_k}}$$

3. **Optional Masking**: If masking is required (e.g., for padded positions in a batch of sequences), apply the mask by setting the scores for masked positions to a very large negative value, ensuring that they have minimal impact after the softmax step.

4. **Softmax**: Apply the softmax function to the scaled scores along each row. This step converts the scores into probabilities, indicating the importance of each key relative to each query.

   $$\text{Attention Weights} = \text{softmax}(\text{Scaled Score Matrix})$$

5. **Output Calculation**: Multiply the attention weights by the value vectors to obtain the final output. This step computes a weighted average of the value vectors, where the weights are determined by the attention scores.

   $$\text{Output} = \text{Attention Weights} \times V$$

#### Implementation Tips
- **Dimensionality**: Ensure the dimensions of your matrices are correct. Matrix multiplication will not be possible if the inner dimensions do not match.
- **Numerical Stability**: When implementing the softmax function, ensure numerical stability by subtracting the maximum value in each row of the scores matrix before applying the exponential function.
- **Batch Processing**: If implementing attention in batch, include an additional batch dimension in your matrices (e.g., $Q\in\mathbb{R}^{B\times T\times d_k}$ for a batch size of $B$) and ensure your implementation supports this.
- **Testing**: Verify the correctness of your implementation with simple test cases to ensure it behaves as expected.

This framework should provide a clear structure for students to implement scaled dot product attention, enhancing their understanding of its role and functionality in self-attention models.

# Task: Please implement a scaled dot product function

In [3]:
def scaled_dot_product(q, k, v, mask=None):
  score_matrix = q @ k.transpose(-2, -1)
  scaled_score = score_matrix / math.sqrt(k.shape[1])
  attention_weight = F.softmax(scaled_score, dim= 1)
  output = attention_weight @ v
  return output

q=torch.Tensor([[-4.682967185974121, -0.46032536029815674, 1.9287296533584595, 1.098872423171997], [1.813373327255249, -0.17624400556087494, -7.465083122253418, 4.692303657531738], [-11.090826034545898, 7.349782943725586, 4.164590835571289, -4.623814582824707]])
k=torch.Tensor([[52.20319747924805, 39.19321060180664, 4.55007791519165, 32.2530403137207], [61.92286682128906, -44.482208251953125, -35.478302001953125, -68.6395263671875], [20.798810958862305, -43.60276412963867, 8.565412521362305, 12.54694938659668]])
v=torch.Tensor([[0.6951367855072021, -0.21053913235664368, 1.9876152276992798, 0.10447879880666733], [0.9846767783164978, 0.6022341847419739, -0.6896607279777527, -1.6564579010009766], [-0.7948723435401917, 0.6899239420890808, -1.8456658124923706, 0.6393752098083496]])
scaled_dot_product(q,k,v)

tensor([[-0.7949,  0.6899, -1.8457,  0.6394],
        [ 0.6951, -0.2105,  1.9876,  0.1045],
        [ 0.6951, -0.2105,  1.9876,  0.1045]])